In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import urllib.request as ur
import json
import datetime
from datetime import datetime
%matplotlib inline

In [2]:
df = pd.read_csv('Data/Argentina.csv')

In [3]:
df.head()

,location,report_date,location_type,country,value,province,county,city,latitude,longitude
0,Argentina-Buenos_Aires,2016-03-19,province,Argentina,130,Buenos Aires,None,None,-34.607568,-58.437089
1,Argentina-Buenos_Aires,2016-03-26,province,Argentina,148,Buenos Aires,None,None,-34.607568,-58.437089
2,Argentina-Buenos_Aires,2016-04-02,province,Argentina,190,Buenos Aires,None,None,-34.607568,-58.437089
3,Argentina-Buenos_Aires,2016-04-09,province,Argentina,199,Buenos Aires,None,None,-34.607568,-58.437089
4,Argentina-Buenos_Aires,2016-04-16,province,Argentina,207,Buenos Aires,None,None,-34.607568,-58.437089


In [4]:
 from datetime import date
# Get Weather related columns from https://www.wunderground.com/ using api: https://api.darksky.net
def get_weather(date, df_row):
    date = datetime.strptime(date, '%Y-%m-%d') 
    
    unixtime = int(time.mktime(date.timetuple()))
    url_fmt = 'https://api.darksky.net/forecast/a5ecdebee6e6b013a7a75790772e6408/{},{},{}?exclude=currently,minutely,hourly,flags'
    #url_fmt = 'https://api.darksky.net/forecast/413fed8c857e15f2f76a7fccfa39b6ab/{},{},{}?exclude=currently,minutely,hourly,flags'
  
    url = url_fmt.format(df_row['latitude'], df_row['longitude'],unixtime)
    f = ur.urlopen(url)

    js = json.loads(f.read())
    
    f.close()
    time.sleep(1.0)
    try:
        #print(js['daily']['data'][0])
        tempMax = js['daily']['data'][0]['temperatureHigh']
        tempLow = js['daily']['data'][0]['temperatureLow']
        dewPoint = js['daily']['data'][0]['dewPoint']
        precip = js['daily']['data'][0]['precipIntensity']
        wind = js['daily']['data'][0]['windSpeed']
    except:
        return None
    
    return str(tempMax) + ' ' + str(tempLow) + ' ' + str(dewPoint) + ' ' + str(precip) + ' ' + str(wind)

In [5]:
# test weather api
print(get_weather(df.loc[30]['report_date'], df.loc[1]))

76.75 63.36 66.07 0.004 5.81


In [6]:
# get weather data
df['weather'] =  df.apply(lambda r: get_weather(r['report_date'], r), axis=1)

In [10]:
# split/format new columns
df[['max_temp','min_temp','dew_point','precipitation','wind']] = df['weather'].str.split(' ', expand=True)
df['max_temp'] = df['max_temp'].astype(float)
df['min_temp'] = df['min_temp'].astype(float)
df['dew_point'] = df['dew_point'].astype(float)
df['precipitation'] = df['precipitation'].astype(float)
df['wind'] = df['wind'].astype(float)

In [11]:
df = df.drop(['weather'],axis=1)  

In [12]:
# save csv file for future needs
df.to_csv('Argentina_weather.csv', encoding='utf-8', index=False)

In [13]:
df.head()

,location,report_date,location_type,country,value,province,county,city,latitude,longitude,max_temp,min_temp,dew_point,precipitation,wind
0,Argentina-Buenos_Aires,2016-03-19,province,Argentina,130,Buenos Aires,None,None,-34.607568,-58.437089,69.97,60.83,48.97,0.0002,13.82
1,Argentina-Buenos_Aires,2016-03-26,province,Argentina,148,Buenos Aires,None,None,-34.607568,-58.437089,72.43,63.38,60.02,0.0000,12.01
2,Argentina-Buenos_Aires,2016-04-02,province,Argentina,190,Buenos Aires,None,None,-34.607568,-58.437089,76.75,63.36,66.07,0.0040,5.81
3,Argentina-Buenos_Aires,2016-04-09,province,Argentina,199,Buenos Aires,None,None,-34.607568,-58.437089,70.23,57.67,59.68,0.0004,10.78
4,Argentina-Buenos_Aires,2016-04-16,province,Argentina,207,Buenos Aires,None,None,-34.607568,-58.437089,78.63,63.30,64.60,0.0000,8.57
